In [17]:
import os
import pandas as pd
import cv2
import numpy as np

CSV_PATH = "/content/imagedata/drive_log.csv"
DATA_DIR_1 = "/content/imagedata"

df = pd.read_csv(CSV_PATH)
print(f"🔍 CSV 로드 완료: {len(df)}개 항목")

def get_actual_path(frame_path):
    if frame_path.startswith("data/"):
        # data1 -> data
        relative_path = frame_path.replace("data/", "")
        return os.path.join(DATA_DIR_1, relative_path)
    else:
        # 그 외는 그냥 기본 데이터 폴더에 붙임
        return os.path.join(DATA_DIR_1, frame_path)

def load_data(df):
    images = []
    labels = []
    label_map = {"left": 0, "center": 1, "right": 2, "stop": 3}

    for idx, row in df.iterrows():
        img_path = get_actual_path(row['frame'])
        if not os.path.exists(img_path):
            print(f"이미지 없음: {img_path}")
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"이미지 읽기 실패: {img_path}")
            continue

        img = cv2.resize(img, (64, 64))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)

        label_str = str(row['label']).lower().strip()
        label = label_map.get(label_str, -1)
        labels.append(label)

    return np.array(images), np.array(labels, dtype=int)

X, y = load_data(df)

valid_idx = y >= 0
X = X[valid_idx]
y = y[valid_idx]

print(f"\n✅ 최종 로딩 완료")
print(f"🖼️  이미지 수: {len(X)}")
print(f"📏  이미지 형태: {X.shape}")
print(f"🏷️  라벨 형태: {y.shape}")
print(f"🔢  라벨 분포: {np.bincount(y)}")


🔍 CSV 로드 완료: 922개 항목

✅ 최종 로딩 완료
🖼️  이미지 수: 922
📏  이미지 형태: (922, 64, 64, 3)
🏷️  라벨 형태: (922,)
🔢  라벨 분포: [186 514 188  34]


In [18]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

# 경로 설정
CSV_PATH = "/content/imagedata/drive_log.csv"
DATA_DIR_1 = "/content/imagedata"    # data1 대신 data 폴더

# 1) 이미지 경로 매핑 함수
def get_actual_path(frame_path):
    if frame_path.startswith("data/"):
        relative_path = frame_path.replace("data/", "")
        return os.path.join(DATA_DIR_1, relative_path)
    else:
        # 예외 처리: 그냥 DATA_DIR_1에 붙임
        return os.path.join(DATA_DIR_1, frame_path)

# 2) 커스텀 데이터셋 클래스
class DriveDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        import pandas as pd
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.label_map = {"left": 0, "center": 1, "right": 2, "stop": 3}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        frame_path = self.data.iloc[idx, 0]  # 'frame' 컬럼
        img_path = get_actual_path(frame_path)

        image = Image.open(img_path).convert("RGB")

        label_str = str(self.data.iloc[idx, 2]).lower().strip()
        label = self.label_map.get(label_str, -1)
        if label == -1:
            raise ValueError(f"Invalid label '{label_str}' at index {idx}")

        if self.transform:
            image = self.transform(image)
        return image, label

# 3) 이미지 전처리 + Augmentation
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 4) 데이터셋 생성 및 분할
dataset = DriveDataset(csv_file=CSV_PATH, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

# 5) Dropout 추가 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(32 * 16 * 16, 64)
        self.fc2 = nn.Linear(64, 4)  # 4 클래스

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = SimpleCNN()
print("✅ 모델 준비 완료!")


✅ 모델 준비 완료!


In [19]:
import torch.optim as optim

# GPU 사용 가능하면 GPU로, 아니면 CPU로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 손실함수, 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
for epoch in range(1, num_epochs + 1):
    # --- 학습 모드 ---
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= total
    train_acc = correct / total

    # --- 평가 모드 ---
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss /= total
    val_acc = correct / total

    print(f"Epoch [{epoch}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

torch.save(model.state_dict(), 'model.pth')
print("✅ 모델이 model.pth로 저장되었습니다.")

Epoch [1/50] Train Loss: 1.0874, Train Acc: 0.5400 Val Loss: 0.9827, Val Acc: 0.5622
Epoch [2/50] Train Loss: 0.9945, Train Acc: 0.5848 Val Loss: 0.9791, Val Acc: 0.5838
Epoch [3/50] Train Loss: 0.9465, Train Acc: 0.5794 Val Loss: 0.9164, Val Acc: 0.5892
Epoch [4/50] Train Loss: 0.8422, Train Acc: 0.6106 Val Loss: 0.7672, Val Acc: 0.6919
Epoch [5/50] Train Loss: 0.7307, Train Acc: 0.6825 Val Loss: 0.5927, Val Acc: 0.7676
Epoch [6/50] Train Loss: 0.5253, Train Acc: 0.7883 Val Loss: 0.4173, Val Acc: 0.8973
Epoch [7/50] Train Loss: 0.3655, Train Acc: 0.8643 Val Loss: 0.2762, Val Acc: 0.9351
Epoch [8/50] Train Loss: 0.2519, Train Acc: 0.9172 Val Loss: 0.1743, Val Acc: 0.9730
Epoch [9/50] Train Loss: 0.2062, Train Acc: 0.9308 Val Loss: 0.1468, Val Acc: 0.9838
Epoch [10/50] Train Loss: 0.1551, Train Acc: 0.9471 Val Loss: 0.1477, Val Acc: 0.9730
Epoch [11/50] Train Loss: 0.1594, Train Acc: 0.9389 Val Loss: 0.1042, Val Acc: 0.9892
Epoch [12/50] Train Loss: 0.1296, Train Acc: 0.9579 Val Loss: 0

In [ ]:
torch.save(model.state_dict(), 'New_model.pth')
print("✅ 모델이 model.pth로 저장되었습니다.")

✅ 모델이 model.pth로 저장되었습니다.


In [ ]:
!pip install pytesseract

In [ ]:
!sudo apt update
!sudo apt install -y tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,305 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,561 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,747 kB]
Hit:13 https://ppa.launchpadcontent.net/ubun

In [16]:
!unzip /content/drive/MyDrive/imagedata.zip

Archive:  /content/drive/MyDrive/imagedata.zip
   creating: imagedata/
  inflating: imagedata/drive_log.csv  
  inflating: imagedata/frame_00000.jpg  
  inflating: imagedata/frame_00001.jpg  
  inflating: imagedata/frame_00002.jpg  
  inflating: imagedata/frame_00003.jpg  
  inflating: imagedata/frame_00004.jpg  
  inflating: imagedata/frame_00005.jpg  
  inflating: imagedata/frame_00006.jpg  
  inflating: imagedata/frame_00007.jpg  
  inflating: imagedata/frame_00008.jpg  
  inflating: imagedata/frame_00009.jpg  
  inflating: imagedata/frame_00010.jpg  
  inflating: imagedata/frame_00011.jpg  
  inflating: imagedata/frame_00012.jpg  
  inflating: imagedata/frame_00013.jpg  
  inflating: imagedata/frame_00014.jpg  
  inflating: imagedata/frame_00015.jpg  
  inflating: imagedata/frame_00016.jpg  
  inflating: imagedata/frame_00017.jpg  
  inflating: imagedata/frame_00018.jpg  
  inflating: imagedata/frame_00019.jpg  
  inflating: imagedata/frame_00020.jpg  
  inflating: imagedata/frame_